In [ ]:
from pytest_cases import case

import numpy as np
import interpax

from jax import numpy as jnp
from viperleed_jax import rfactor

from matplotlib import pyplot as plt

from viperleed.calc.files.iorfactor import beamlist_to_array
from viperleed.calc.files.beams import readOUTBEAMS

from viperleed_jax.tensor_calculator import interpolate_ragged_array

In [ ]:
_MOCK_ROUGH_E_AXIS = np.arange(15, 300, 3)
_MOCK_FINE_E_AXIS = np.arange(15, 300, 0.5)

_MOCK_SIN_INTENSITY = np.sin(_MOCK_ROUGH_E_AXIS / (3 * np.pi)) ** 2
_MOCK_COS_INTENSITY = np.cos(_MOCK_ROUGH_E_AXIS / (3 * np.pi)) ** 2
_MOCK_CONST_INTENSITY = np.ones_like(_MOCK_ROUGH_E_AXIS)

In [ ]:
sin_spline = interpax.CubicSpline(_MOCK_ROUGH_E_AXIS, _MOCK_SIN_INTENSITY)
cos_spline = interpax.CubicSpline(_MOCK_ROUGH_E_AXIS, _MOCK_COS_INTENSITY)
const_spline = interpax.CubicSpline(_MOCK_ROUGH_E_AXIS, _MOCK_CONST_INTENSITY)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(
    _MOCK_ROUGH_E_AXIS,
    sin_spline(_MOCK_ROUGH_E_AXIS),
    label='sin',
    linestyle='',
    marker='x',
)
plt.plot(_MOCK_FINE_E_AXIS, sin_spline(_MOCK_FINE_E_AXIS), label='sin')
plt.show()

In [ ]:
exp_out_beams = readOUTBEAMS(
    '/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/Fe2O3_012/converged/EXPBEAMS.csv'
)
exp_energies, _, _, exp_beams = beamlist_to_array(exp_out_beams)
exp_spline = interpolate_ragged_array(exp_energies, exp_beams)
exp_hk = [b.hk for b in exp_out_beams]

In [ ]:
theo_out_beams = readOUTBEAMS(
    '/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/Fe2O3_012/converged/OUT/THEOBEAMS.csv'
)
theo_hk = [b.hk for b in theo_out_beams]
theo_out_dict = {b.hk: b for b in theo_out_beams}
sorted_theo = [theo_out_dict[hk] for hk in exp_hk]
theo_energies, _, _, theo_beams = beamlist_to_array(sorted_theo)
theo_spline = interpolate_ragged_array(theo_energies - 1.0, theo_beams)

In [ ]:
rfactor.pendry_R(theo_spline, 5.0, 0.5, theo_energies, exp_spline)

In [ ]:
plt.imshow(exp_spline.c[1], aspect='auto')

In [ ]:
exp_spline.c

In [ ]:
beam = 11
exp = exp_beams[:, beam]
theo = theo_beams[:, beam]
plt.plot(theo_energies, theo / np.nanmax(theo), label='theo')
plt.plot(exp_energies, exp / np.nanmax(exp), label='exp')

In [ ]:
exp_energies

In [ ]:
theo_energies

In [ ]:
exp = exp_spline(theo_energies)[:, beam]
theo = theo_spline(theo_energies)[:, beam]
plt.plot(theo_energies, theo / np.nanmax(theo), label='theo')
plt.plot(theo_energies, exp / np.nanmax(exp), label='exp')

In [ ]:
theo_beams

In [ ]:
theo_spline(theo_energies)[:, 5]

In [ ]:
plt.plot(theo_energies, theo_beams[:, 25] / np.nanmax(theo_beams[:, 25]))

In [ ]:
theo_beams.shape, exp_beams.shape

In [ ]:
plt.plot(theo_energies, exp_spline(theo_energies), label='exp')

In [ ]:
rfactor.pendry_R(sin_spline, 3.0, 0.5, _MOCK_FINE_E_AXIS, sin_spline)

In [ ]:
rfactor.pendry_R(cos_spline, 3.0, 0.5, _MOCK_FINE_E_AXIS, sin_spline)

In [ ]:
rfactor.pendry_R(const_spline, 3.0, 0.5, _MOCK_FINE_E_AXIS, sin_spline)

In [ ]:
plt.plot(np.sin(_MOCK_E_AXIS / (3 * np.pi)) ** 2)

In [ ]:
plt.plot(np.cos(_MOCK_E_AXIS / (3 * np.pi)) ** 2)